In [1]:
%load_ext autoreload
%autoreload 2

In [1]:
import torch
import torch.nn as nn

## Get weights

Proposed API

```python
ResNet.resnet50(pretrained=True, dataset='imagenet')

ResNet.pretrained_models
```



In [2]:
import requests

def download_file_from_google_drive(id, destination):
    URL = "https://docs.google.com/uc?export=download"

    session = requests.Session()

    response = session.get(URL, params = { 'id' : id }, stream = True)
    token = get_confirm_token(response)

    if token:
        params = { 'id' : id, 'confirm' : token }
        response = session.get(URL, params = params, stream = True)

    save_response_content(response, destination)    

def get_confirm_token(response):
    for key, value in response.cookies.items():
        if key.startswith('download_warning'):
            return value

    return None

def save_response_content(response, destination):
    CHUNK_SIZE = 1024

    with open(destination, "wb") as f:
        for chunk in response.iter_content(CHUNK_SIZE):
            if chunk: # filter out keep-alive new chunks
                f.write(chunk)

In [3]:
download_file_from_google_drive('1CMyib_ACsWUIbXa7KjX3NXKkfAQyNnLd', '/home/zuppif/Desktop/test.pth')

In [7]:
from glasses.nn.models import *


model = ResNet.resnet18(pretrained=True)



INFO:root:Loaded resnet18 pretrained weights.


In [ ]:
import torch
from glasses.nn.models import *
x = torch.rand((1,3,224,224))


FishNet.fishnet99()(x)

In [ ]:
import torch
from glasses.nn.models import *
x = torch.rand((1,3,224,224))


FishNet.fishnet99()(x)

In [ ]:
from glasses.nn.blocks import Conv2dPad

x = torch.rand((1,32,9,9))

Conv2dPad(32, 64, kernel_size=9, mode = 'auto', padding=0)

In [ ]:
from glasses.nn.models import *

model =  ResNetXt.resnet18(pretrained=True)

In [ ]:
model.encoder.blocks[-1]

In [ ]:
import timm

timm.list_models('vgg')

In [ ]:
import timm
m = timm.create_model('seresnet101', pretrained=True)
m

In [ ]:
from dataclasses import dataclass
from typing import Tuple, List
from pprint import pprint
from benchmark import benchmark
from glasses.utils.ModuleTransfer import ModuleTransfer
from functools import partial
from glasses.nn.models import *
from torchvision.models import resnet18, resnet34, resnet50, resnet101, resnet152, resnext50_32x4d, resnext101_32x8d, wide_resnet50_2, wide_resnet101_2
from tqdm.autonotebook import tqdm
import torchvision.transforms as T
import torch.nn as nn
import torch
from PIL import Image
from glasses.utils.PretrainedWeightsProvider import Config


models_weights = {
    'seresnet50': Config(),
#     'efficientnet_b4': Config(resize=380, input_size=380, interpolation='bicubic')
}

zoo_models_mapping = {
    'seresnet50': [partial(timm.create_model, 'seresnet101',pretrained=True), SEResNet.cse_resnet34],
   
#   'efficientnet_b4': [partial(timm.create_model, 'tf_efficientnet_b4', pretrained=True), EfficientNet.efficientnet_b4]
}



# pprint(models_weights)

In [ ]:
def clone_model(src: nn.Module, dst: nn.Module) -> nn.Module:
    src = src.eval()
    dst = dst.eval()

    x = torch.rand((1, 3, 224, 224))
    a = src(x)
    b = dst(x)

    # assert not torch.equal(a, b)

    ModuleTransfer(src, dst)(x)

    # a = src(x)
    # b = dst(x)

    # assert torch.equal(a, b)

    return dst


In [ ]:
bar = tqdm(zoo_models_mapping.items())

for name, mapping in bar:
    
    src, dst = mapping[0](), mapping[1]()
    tr = models_weights[name].transform
    print(tr)
    clone_model(src, dst)
    bar.set_description(name)
    res = benchmark(dst.cuda(), tr)
    print(name, res)
    
    res = benchmark(src.cuda(), tr)
    print(name, res)

In [ ]:
[{'original_time': 84.83511066436768,████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▌| 49866/50000 [00:05<00:00, 9220.38it/s]
  'original_top1': 0.74654,
  'original_top5': 0.91924},
 {'original_time': 122.87314462661743,
  'original_top1': 0.74602,
  'original_top5': 0.9159},
 {'original_time': 168.03247451782227,
  'original_top1': 0.79398,
  'original_top5': 0.94576},
 {'original_time': 281.6872932910919,
  'original_top1': 0.81218,
  'original_top5': 0.95528},
 {'original_time': 588.1660254001617,
  'original_top1': 0.8279,
  'original_top5': 0.96234},
 {'original_time': 1126.7628757953644,
  'original_top1': 0.8346,
  'original_top5': 0.96624}]

In [ ]:
EfficientNet.b0()

In [ ]:
from efficientnet_pytorch import EfficientNet as EfficientNetPytorch
from glasses.nn.models import  EfficientNet
from torchsummary import summary



summary(EfficientNet.b0().cuda(), (3, 224, 224))

In [ ]:
model = EfficientNetPytorch.from_pretrained('efficientnet-b0')
model.set_swish(False)
summary(model.cuda(), (3, 224, 224))

In [ ]:
model